A função para decidir se é preciso trocar de caixa ou não é a seguinte:

In [127]:
# The probability to win will always be higher if the player switches boxes, unless there is only one box to be opened,
# in which case, the remaining box is the one with the Apple Watch, but this is not a realistic scenario since the 
# player will be sure the watch is there.
# Fore more information, watch https://www.youtube.com/watch?v=LIyg28vXQdQ

def switch_boxes(total_boxes, open_boxes=0):

  if (total_boxes < open_boxes + 2):
    print("The game needs at least two closed boxes, one with the Apple Watch and one empty, to make sense.")
    return False
  
  p_win_before_open = 1 / total_boxes

  p_win_if_switch_after_open = (total_boxes - 1) / (total_boxes * (total_boxes - open_boxes - 1))
  
  # if p_win_if_switch_after_open == p_win_before_open, probabilistically speaking it doesn't make any difference
  # to switch the currently selected box or not. In this implementation, I decided to switch.
  if p_win_if_switch_after_open >= p_win_before_open:
    return True
  return False


Acontece que o modelo matemático que descreve este problema se chama problema de **Monty Hall**, e de acordo com a sua solução, é sempre vantajoso trocar de caixa após uma ou mais caixas vazias serem reveladas. Vejamos alguns exemplos.

In [128]:
print(switch_boxes(7, 5))

True


In [129]:
print(switch_boxes(3, 1))

True


In [130]:
print(switch_boxes(50, 1))

True


In [131]:
print(switch_boxes(50, 48))

True


In [132]:
print(switch_boxes(978787, 3574))

True


In [133]:
print(switch_boxes(3, 0))

True


Abaixo, segue um programa que simula um problema de Monty Hall iterativo. O jogador e o condutor do jogo alternam entre decidir trocar de caixa e revelar uma caixa vazia, respectivamente.

É possível variar o número de caixas (variável **number_of_boxes**) para executar simulações mais longas, mas como vimos na função passada, independente do número de caixas presentes no jogo e quantas já foram reveladas, é sempre probabilisticamente vantajoso mudar sua escolha de caixa após pelo menos uma caixa vazia ser revelada.

In [134]:
import sys
import random

random.seed()
number_of_boxes = int(input("Number of boxes in the game: "))

# Generate random boxes contents
boxes = ["empty box" for i in range(number_of_boxes - 1)] + ["apple watch"]

# Shuffle boxes
random.shuffle(boxes)
print("Boxes (in position): {}".format(boxes))

# Player pick first box
box_choice = random.randrange(number_of_boxes)
print("First box selected by the player: {}".format(box_choice))

# Initialize common auxiliary variables
apple_watch_position = boxes.index("apple watch")
print("Apple watch position: {}".format(apple_watch_position))

Number of boxes in the game: 50
Boxes (in position): ['empty box', 'empty box', 'empty box', 'apple watch', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box', 'empty box']
First box selected by the player: 41
Apple watch position: 3


In [135]:
# Slightly slower solution:
# computing the next boxes to reveal and to swap to does require attempts, 
# but consumes less memory than the alternative with the available_boxes list
# -----------------------------------------------------------------------------
#
# Initialize solution-specific auxiliary variables
revealed_boxes = []
box_to_reveal = random.randrange(number_of_boxes)
box_to_swap = random.randrange(number_of_boxes)

while len(revealed_boxes) < number_of_boxes - 2:

  # Host reveals a random empty box
  while box_to_reveal == box_choice or box_to_reveal in revealed_boxes or box_to_reveal == apple_watch_position:
    box_to_reveal = random.randrange(number_of_boxes)

  revealed_boxes.append(box_to_reveal)
  print("Host revealed the empty box #{}".format(box_to_reveal))

  # Player swap to another random, unrevealed box
  # According to the Monty Hall problem, a player should always switch boxes after the host reveals empty boxes.
  # This if statement is here only to additionally illustrate this behavior (no matter how many times this 
  # code runs, or how many boxes we set on the number_of_boxes variable, the else block will never run).
  if (switch_boxes(number_of_boxes, len(revealed_boxes))):
    while box_to_swap == box_choice or box_to_swap in revealed_boxes:
      box_to_swap = random.randrange(number_of_boxes)

    box_choice = box_to_swap
    print("Player swapped his/her box to #{}".format(box_to_swap))
  else:
    print("Player did not swap boxes")
  
if boxes[box_choice] == "apple watch":
    print("You won the Apple Watch!")
else:
    print("You won nothing.")

Host revealed the empty box #42
Player swapped his/her box to #1
Host revealed the empty box #38
Player swapped his/her box to #9
Host revealed the empty box #15
Player swapped his/her box to #8
Host revealed the empty box #7
Player swapped his/her box to #20
Host revealed the empty box #9
Player swapped his/her box to #23
Host revealed the empty box #34
Player swapped his/her box to #26
Host revealed the empty box #17
Player swapped his/her box to #35
Host revealed the empty box #25
Player swapped his/her box to #16
Host revealed the empty box #28
Player swapped his/her box to #20
Host revealed the empty box #29
Player swapped his/her box to #22
Host revealed the empty box #31
Player swapped his/her box to #0
Host revealed the empty box #5
Player swapped his/her box to #49
Host revealed the empty box #19
Player swapped his/her box to #39
Host revealed the empty box #8
Player swapped his/her box to #22
Host revealed the empty box #45
Player swapped his/her box to #30
Host revealed the 

E uma versão em média um pouco mais rápida, mas que consome mais memória:



In [136]:
# Faster Solution:
# computing the next boxes to reveal and to swap to does not require attempts, 
# but it consumes more memory because of the available_boxes list.
# -----------------------------------------------------------------------------
#
# Initialize solution-specific auxiliary variables
revealed_boxes = []
available_boxes = [ i for i in range(len(boxes)) if i != apple_watch_position and i != box_choice ]

while len(revealed_boxes) < number_of_boxes - 2:

  # Host reveals a random empty box
  box_to_reveal = random.choice(available_boxes)
  revealed_boxes.append(box_to_reveal)
  available_boxes.remove(box_to_reveal)

  print("Host revealed the empty box #{}".format(box_to_reveal))

  # Player swap to another random, unrevealed box.
  # According to the Monty Hall problem, a player should always switch boxes after the host reveals empty boxes
  # This if statement is here only to additionally illustrate this behavior (no matter how many times this 
  # code runs, or how many boxes we set on the number_of_boxes variable, the else block will never run).
  if (switch_boxes(number_of_boxes, len(revealed_boxes))):
    box_to_swap = random.choice(available_boxes + [ apple_watch_position ])
    
    if box_choice != apple_watch_position:
      available_boxes.append(box_choice)

    if box_to_swap != apple_watch_position:
      available_boxes.remove(box_to_swap)

    box_choice = box_to_swap
    print("Player swapped his/her box to #{}".format(box_to_swap))
  else:
    print("Player did not swap boxes")
  
if box_choice == apple_watch_position:
    print("You won the Apple Watch!")
else:
    print("You won nothing.")


Host revealed the empty box #47
Player swapped his/her box to #14
Host revealed the empty box #5
Player swapped his/her box to #0
Host revealed the empty box #39
Player swapped his/her box to #35
Host revealed the empty box #27
Player swapped his/her box to #33
Host revealed the empty box #7
Player swapped his/her box to #24
Host revealed the empty box #49
Player swapped his/her box to #4
Host revealed the empty box #32
Player swapped his/her box to #17
Host revealed the empty box #15
Player swapped his/her box to #3
Host revealed the empty box #13
Player swapped his/her box to #40
Host revealed the empty box #43
Player swapped his/her box to #48
Host revealed the empty box #6
Player swapped his/her box to #30
Host revealed the empty box #22
Player swapped his/her box to #25
Host revealed the empty box #10
Player swapped his/her box to #26
Host revealed the empty box #2
Player swapped his/her box to #20
Host revealed the empty box #41
Player swapped his/her box to #31
Host revealed the

Por fim, como último experimento, avaliaremos o caso genérico, onde existem **n** caixas no jogo, e o condutor do jogo decide abrir **g** de uma vez. Para 1000 **n** e **g** diferentes e aleatórios, é possível observar que em todos é vantajoso trocar a caixa selecionada.

In [137]:
def compute_random_scenario(max_number_of_boxes):
  n = random.randrange(2, max_number_of_boxes + 1)
  g = random.randrange(n - 1)
  return switch_boxes(n, g)

max_number_of_boxes = int(input("Maximum number of boxes in the game: "))
number_of_simulations = int(input("Number of simulations: "))
results = []

for i in range(number_of_simulations):
  results.append(compute_random_scenario(max_number_of_boxes))


Maximum number of boxes in the game: 50
Number of simulations: 1000


In [138]:
list(filter(lambda result: result != True, results))

[]